In [39]:
import pandas as pd
import numpy as np
import json

DATA_IN_PATH='./data_in/'
DATA_OUT_PATH='./data_out/'

INPUT_TRAIN_DATA_FILE_NAME='train_input.npy'
LABEL_TRAIN_DATA_FILE_NAME='train_label.npy'
DATA_CONFIGS_FILE_NAME='data_configs.json'

input_data=np.load(open(DATA_IN_PATH+INPUT_TRAIN_DATA_FILE_NAME,'rb'))
label_data=np.load(open(DATA_IN_PATH+LABEL_TRAIN_DATA_FILE_NAME,'rb'))

prepro_config=None

with open(DATA_IN_PATH+DATA_CONFIGS_FILE_NAME,'r') as f:
    prepro_configs=json.load(f)

In [40]:
from sklearn.model_selection import train_test_split

TEST_SPLIT=0.1
RANDOM_SEED=13371447

train_input, test_input, train_label, test_label=train_test_split(input_data,label_data,test_size=TEST_SPLIT,random_state=RANDOM_SEED)

In [41]:
import tensorflow as tf

BATCH_SIZE=16
NUM_EPOCHS=3

def mapping_fn(X, Y):
    inputs, labels={'x':X},Y
    return inputs, labels

def train_input_fn():
    dataset=tf.data.Dataset.from_tensor_slices((train_input, train_label))
    dataset=dataset.shuffle(buffer_size=1000)
    dataset=dataset.batch(BATCH_SIZE)
    
    dataset=dataset.map(mapping_fn)
    
    dataset=dataset.repeat(count=NUM_EPOCHS)
    
    iterator=tf.compat.v1.data.make_one_shot_iterator(dataset)
    #dataset.make_one_shot_iterator()

    return iterator.get_next()

def eval_input_fn():
    dataset=tf.data.Dataset.from_tensor_slices((test_input,test_label))
    dataset=dataset.map(mapping_fn)
    dataset=dataset.batch(BATCH_SIZE)
    iterator=dataset.make_one_shot_iterator()
    
    return iterator.get_next()

In [42]:
VOCAB_SIZE=prepro_configs['vocab_size']

In [43]:
WORD_EMBEDDING_DIM=100
HIDDEN_STATE_DIM=150
DENSE_FEATURE_DIM=150

learning_rate=0.001

In [44]:
def model_fn(features, labels, mode):
    TRAIN=mode==tf.estimator.ModeKeys.TRAIN
    EVAL=mode==tf.estimator.ModeKeys.EVAL
    PREDICT=mode==tf.estimator.ModeKeys.PREDICT
    
    embedding_layer=tf.keras.layers.Embedding(VOCAB_SIZE,WORD_EMBEDDING_DIM)(features['x'])
    
    embedding_layer=tf.keras.layers.Dropout(0.2)(embedding_layer)
    
    rnn_layers=[tf.nn.rnn_cell.LSTMCell(size) for size in [HIDDEN_STATE_DIM, HIDDEN_STATE_DIM]]
    
    multi_rnn_cell=tf.nn.rnn_cell.MultiRNNCell(rnn_layers)
    
    outputs,state=tf.nn.dynamic_rnn(cell=multi_tnn_cell,inputs=embedding_layer,dtype=tf.float32)
    
    outputs=tf.keras.layers.Dropout(0.2)(outputs)
    hidden_layer=tf.keras.layers.Dense(DENSE_FEATURE_DIM,activation=tf.nn.tanh)(outputs[:,-1,:])
    hidden_layer=tf.keras.layers.Dropout(0.2)(hidden_layer)
    logits=tf.keras.layers.Dense(1)(hidden_layer)
    logits=tf.squeeze(logits, axis=-1)
    
    sigmoid_logits=tf.nn.sigmoid(logits)
    
    if PREDICT:
        predictions={'sentiment':sigmoid_logits}
        
        return tf.estimator.EstimatorSpec(mode=mode,predictions=predictions)
    
    loss=tf.losses.sigmoid_cross_entropy(labels, logits)
    
    if EVAL:
        accuracy=tf.metrics.accuracy(labels, tf.round(sigmoid_logits))
        eval_metric_ops={'acc':accuracy}
        
        return tf.estimator.EstimatorSpec(mode, loss=loss,eval_metric_ops=eval_metric_ops)
    
    if TRAIN:
        global_step=tf.train.get_global_step()
        train_op=tf.train.AdamOptimizer(learning_rate).minimize(loss, global_step)
        
        return tf.estimator.EstimatorSpec(mode=mode,train_op=train_op,loss=loss)

In [45]:
import os

if not os.path.exists(DATA_OUT_PATH):
    os.makedirs(DATA_OUT_PATH)
    
est=tf.estimator.Estimator(model_fn,model_dir=DATA_OUT_PATH+'checkpoint/rnn')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './data_out/checkpoint/rnn', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [46]:
est.train(train_input_fn)

INFO:tensorflow:Calling model_fn.


AttributeError: module 'tensorflow._api.v2.nn' has no attribute 'rnn_cell'